# DW_pandas_Task7

## 20210102

DataWhale_pandas组队学习第七章，缺失数据，加油！！！

### 关系型连接

In [1]:
import pandas as pd
import numpy as np

### 缺失值的统计和删除

In [2]:
df = pd.read_csv('../data/learn_pandas.csv', usecols = ['Grade', 'Name', 'Gender', 'Height', 'Weight', 'Transfer'])
df.isna().head()

,Grade,Name,Gender,Height,Weight,Transfer
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,True,False,False
4,False,False,False,False,False,False


In [3]:
df.isna().sum()/df.shape[0] # 查看缺失的比例

Grade       0.000
Name        0.000
Gender      0.000
Height      0.085
Weight      0.055
Transfer    0.060
dtype: float64

In [4]:
df[df.Height.isna()].head()

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
12,Senior,Peng You,Female,NaN,48.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N
36,Freshman,Xiaojuan Qin,Male,NaN,79.0,Y
60,Freshman,Yanpeng Lv,Male,NaN,65.0,N


In [5]:
sub_set = df[['Height', 'Weight', 'Transfer']]
df[sub_set.isna().all(1)] # 全部缺失

,Grade,Name,Gender,Height,Weight,Transfer
102,Junior,Chengli Zhao,Male,NaN,NaN,NaN


In [6]:
df[sub_set.isna().any(1)].head() # 至少有一个缺失

,Grade,Name,Gender,Height,Weight,Transfer
3,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N
9,Junior,Juan Xu,Female,164.8,NaN,N
12,Senior,Peng You,Female,NaN,48.0,NaN
21,Senior,Xiaopeng Shen,Male,166.0,62.0,NaN
26,Junior,Yanli You,Female,NaN,48.0,N


In [7]:
df[sub_set.notna().all(1)].head() # 没有缺失

,Grade,Name,Gender,Height,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,158.9,46.0,N
1,Freshman,Changqiang You,Male,166.5,70.0,N
2,Senior,Mei Sun,Male,188.9,89.0,N
4,Sophomore,Gaojuan You,Male,174.0,74.0,N
5,Freshman,Xiaoli Qian,Female,158.0,51.0,N


In [8]:
sub_set.notna().all(1).head()

0     True
1     True
2     True
3    False
4     True
dtype: bool

缺失数据可以使用isna或isnull（两个函数没有区别）来查看每个单元格是否缺失，通过和sum的组合可以计算出每列缺失值的比例，注意一下相关的例子，突然领悟到Series和DataFrame之间的关系运用，还有布尔值和数据之间的转换，做的多，学的多。

In [9]:
res = df.dropna(how = 'any', subset = ['Height', 'Weight'])
res.shape  # 删除身高体重至少有一个缺失的行

(174, 6)

In [10]:
res = df.dropna(1, thresh=df.shape[0]-15) # 身高被删除
res.head()   # 删除超过15个缺失值的列

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


In [11]:
res = df.loc[:, ~(df.isna().sum()>15)]
res.head()   # 不用dropna同样是可行的，例如上述的操作，也可以使用布尔索引来完成

,Grade,Name,Gender,Weight,Transfer
0,Freshman,Gaopeng Yang,Female,46.0,N
1,Freshman,Changqiang You,Male,70.0,N
2,Senior,Mei Sun,Male,89.0,N
3,Sophomore,Xiaojuan Sun,Female,41.0,N
4,Sophomore,Gaojuan You,Male,74.0,N


### 缺失值的填充和插值

In [12]:
s = pd.Series([np.nan, 1, np.nan, np.nan, 2, np.nan], list('aaabcd'))
s

a    NaN
a    1.0
a    NaN
b    NaN
c    2.0
d    NaN
dtype: float64

In [13]:
s.to_frame()   # 学习一下群里的两种转DataFrame方式，作一下对比

,0
a,NaN
a,1.0
a,NaN
b,NaN
c,2.0
d,NaN


In [14]:
s.reset_index()

,index,0
0,a,NaN
1,a,1.0
2,a,NaN
3,b,NaN
4,c,2.0
5,d,NaN


In [15]:
s.fillna(method='ffill') # 用前面的值向后填充

a    NaN
a    1.0
a    1.0
b    1.0
c    2.0
d    2.0
dtype: float64

In [16]:
s.fillna(method='ffill', limit=1) # 连续出现的缺失，最多填充一次

a    NaN
a    1.0
a    1.0
b    NaN
c    2.0
d    2.0
dtype: float64

In [17]:
s.fillna(s.mean()) # value为标量

a    1.5
a    1.0
a    1.5
b    1.5
c    2.0
d    1.5
dtype: float64

In [18]:
s.fillna({'a': 100, 'd': 200}) # 通过索引映射填充的值

a    100.0
a      1.0
a    100.0
b      NaN
c      2.0
d    200.0
dtype: float64

In [19]:
df.groupby('Grade')['Height'].transform(lambda x: x.fillna(x.mean())).head()  # 根据年级进行身高的均值填充

0    158.900000
1    166.500000
2    188.900000
3    163.075862
4    174.000000
Name: Height, dtype: float64

在fillna中有三个参数是常用的：value, method, limit。其中，value为填充值，可以是标量，也可以是索引到元素的字典映射；method为填充方法，有用前面的元素填充ffill和用后面的元素填充bfill两种类型，limit参数表示连续缺失值的最大填充次数。

很多用法都可以随意搭配在一起使用，前提是你要先熟练运用这些语法。

#### 【练一练】
对一个序列以如下规则填充缺失值：如果单独出现的缺失值，就用前后均值填充，如果连续出现的缺失值就不填充，即序列`[1, NaN, 3, NaN, NaN]`填充后为`[1, 2, 3, NaN, NaN]`，请利用`fillna`函数实现。（提示：利用`limit``参数）

In [20]:
f = pd.Series([1, np.nan, 3, np.nan, np.nan], list('12345'))
f

1    1.0
2    NaN
3    3.0
4    NaN
5    NaN
dtype: float64

In [22]:
f.fillna(method='ffill', limit=1)

1    1.0
2    1.0
3    3.0
4    3.0
5    NaN
dtype: float64

做不出来，好象很难的样子，这阶段工作繁忙，学习进度不一定能跟上，学多少算多少，估计挺不过来了，请见谅！